In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
### Ваш код ###
from sklearn.model_selection import train_test_split

X = data.drop('MEDV',axis=1)
y = data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
### Ваш код ###
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score


# линейная модель
model_lr = LinearRegression()
model_lr.fit(X_train,y_train)
lr_pred = model_lr.predict(X_test)

# Ridge
model_ridge = Ridge()
model_ridge.fit(X_train,y_train)
ridge_pred = model_ridge.predict(X_test)

# Lasso
model_lasso = Lasso()
model_lasso.fit(X_train,y_train)
lasso_pred = model_lasso.predict(X_test)

r2_lr = r2_score(y_test, lr_pred)
r2_ridge = r2_score(y_test, ridge_pred)
r2_lasso = r2_score(y_test, lasso_pred)
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:
### Ваш код ###
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV

params = [10**i for i in range(-5, 6)]

# RidgeCV
model_ridge_CV = RidgeCV(alphas = params)
model_ridge_CV.fit(X_train, y_train)
ridgeCV_pred = model_ridge_CV.predict(X_test)

# LassoCV
model_lasso_CV = LassoCV(alphas = params)
model_lasso_CV.fit(X_train, y_train)
lassoCV_pred = model_lasso_CV.predict(X_test)

#GridSearchCV
param_grid = {'alpha': params}
model_ridge_grid = Ridge()
model_lasso_grid = Lasso()
grid_model_ridge = GridSearchCV(estimator=model_ridge_grid, param_grid=param_grid, cv=5)
grid_model_ridge.fit(X_train, y_train)
grid_model_lasso = GridSearchCV(estimator=model_lasso_grid, param_grid=param_grid, cv=5)
grid_model_lasso.fit(X_train, y_train)


r2_ridge_grid_search = grid_model_ridge.best_score_
r2_ridge_cv = r2_score(y_test, ridgeCV_pred)
r2_lasso_grid_search = grid_model_lasso.best_score_
r2_lasso_cv = r2_score(y_test, lassoCV_pred)
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

r2_ridge_cv, r2_lasso_cv, r2_ridge_grid_search, r2_lasso_grid_search

(0.7034284982969798, 0.7034949758152599, 0.712425401352967, 0.7123497706813428)

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.linear_model import RidgeCV, LassoCV

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

# Ridge
model_ridge_standart_scaler = Ridge()
model_ridge_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
ridge_standart_scaler_pred = model_ridge_standart_scaler.predict(X_test_standart_scaler_transformed)

model_ridge_minmax_scaler = Ridge()
model_ridge_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
ridge_minmax_scaler_pred = model_ridge_minmax_scaler.predict(X_test_minmax_scaler_transformed)

# Lasso
model_lasso_standart_scaler = Lasso()
model_lasso_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
lasso_standart_scaler_pred = model_lasso_standart_scaler.predict(X_test_standart_scaler_transformed)

model_lasso_minmax_scaler = Lasso()
model_lasso_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
lasso_minmax_scaler_pred = model_lasso_minmax_scaler.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler = r2_score(y_test, ridge_standart_scaler_pred)
r2_ridge_min_max_scaler = r2_score(y_test, ridge_minmax_scaler_pred)
r2_lasso_standart_scaler = r2_score(y_test, lasso_standart_scaler_pred)
r2_lasso_min_max_scaler = r2_score(y_test, lasso_minmax_scaler_pred)
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

r2_ridge_standart_scaler, r2_ridge_min_max_scaler, r2_lasso_standart_scaler, r2_lasso_min_max_scaler

(0.7032633383504199, 0.7010203723781032, 0.627705201493149, 0.197922380429706)

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

params = [10**i for i in range(-5, 6)]

# RidgeCV
model_ridge_standart_scaler_cv = RidgeCV(alphas = params)
model_ridge_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
ridge_standart_scaler_cv_pred = model_ridge_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_ridge_min_max_scaler_cv = RidgeCV(alphas = params)
model_ridge_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
ridge_min_max_scaler_cv_pred = model_ridge_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

# LassoCV
model_lasso_standart_scaler_cv = LassoCV(alphas = params)
model_lasso_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
lasso_standart_scaler_cv_pred = model_lasso_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_lasso_min_max_scaler_cv = LassoCV(alphas = params)
model_lasso_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
lasso_min_max_scaler_cv_pred = model_lasso_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler_cv = r2_score(y_test, ridge_standart_scaler_cv_pred)
r2_ridge_min_max_scaler_cv = r2_score(y_test, ridge_min_max_scaler_cv_pred)
r2_lasso_standart_scaler_cv = r2_score(y_test, lasso_standart_scaler_cv_pred)
r2_lasso_min_max_scaler_cv = r2_score(y_test, lasso_min_max_scaler_cv_pred)
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

r2_ridge_standart_scaler_cv, r2_ridge_min_max_scaler_cv, r2_lasso_standart_scaler_cv, r2_lasso_min_max_scaler_cv

(0.7011546134235851,
 0.7034888824627393,
 0.7027330685181044,
 0.7034939090973591)

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

#PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.2, random_state=101)

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

# Ridge
model_ridge_standart_scaler = Ridge()
model_ridge_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
ridge_standart_scaler_pred = model_ridge_standart_scaler.predict(X_test_standart_scaler_transformed)

model_ridge_minmax_scaler = Ridge()
model_ridge_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
ridge_minmax_scaler_pred = model_ridge_minmax_scaler.predict(X_test_minmax_scaler_transformed)

# Lasso
model_lasso_standart_scaler = Lasso()
model_lasso_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
lasso_standart_scaler_pred = model_lasso_standart_scaler.predict(X_test_standart_scaler_transformed)

model_lasso_minmax_scaler = Lasso()
model_lasso_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
lasso_minmax_scaler_pred = model_lasso_minmax_scaler.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler_poly = r2_score(y_test, ridge_standart_scaler_pred)
r2_ridge_min_max_scaler_poly = r2_score(y_test, ridge_minmax_scaler_pred)
r2_lasso_standart_scaler_poly = r2_score(y_test, lasso_standart_scaler_pred)
r2_lasso_min_max_scaler_poly = r2_score(y_test, lasso_minmax_scaler_pred)

results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

r2_ridge_standart_scaler_poly, r2_ridge_min_max_scaler_poly, r2_lasso_standart_scaler_poly, r2_lasso_min_max_scaler_poly

(0.8181752651058304,
 0.7966345636020606,
 0.6807930537311699,
 0.24121621257587678)

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

#PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.2, random_state=101)

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

params = [10**i for i in range(-5, 6)]

# RidgeCV
model_ridge_standart_scaler_cv = RidgeCV(alphas = params)
model_ridge_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
ridge_standart_scaler_poly_cv_pred = model_ridge_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_ridge_min_max_scaler_cv = RidgeCV(alphas = params)
model_ridge_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
ridge_min_max_scaler_poly_cv_pred = model_ridge_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

# LassoCV
model_lasso_standart_scaler_cv = LassoCV(alphas = params, max_iter=1000000000)
model_lasso_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
lasso_standart_scaler_poly_cv_pred = model_lasso_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_lasso_min_max_scaler_cv = LassoCV(alphas = params, max_iter=1000000000)
model_lasso_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
lasso_min_max_scaler_poly_cv_pred = model_lasso_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler_poly_cv = r2_score(y_test, ridge_standart_scaler_poly_cv_pred)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, ridge_min_max_scaler_poly_cv_pred)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, lasso_standart_scaler_poly_cv_pred)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, lasso_min_max_scaler_poly_cv_pred)

results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

r2_ridge_standart_scaler_poly_cv, r2_ridge_min_max_scaler_poly_cv, r2_lasso_standart_scaler_poly_cv, r2_lasso_min_max_scaler_poly_cv

(0.8181752651038463,
 0.8258914935703197,
 0.8304299177817691,
 0.8134437022886388)

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning

import warnings

warnings.simplefilter("ignore", category=ConvergenceWarning)

In [12]:
pipeline = Pipeline([('imputer', StandardScaler()),
                     ('polynomial', PolynomialFeatures()),
                     ('regression', Lasso())])
parameters = {'imputer':[StandardScaler(), MinMaxScaler()],
              'polynomial__degree':np.arange(1,3,1),
              'regression':[Lasso(), Ridge()],
              'regression__alpha':np.logspace(-2, 2, 5)}
select_model = GridSearchCV(pipeline, parameters, scoring='r2', n_jobs=-1, verbose=10 )
select_model.fit(X_train, y_train)
select_model.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits


Pipeline(steps=[('imputer', MinMaxScaler()),
                ('polynomial', PolynomialFeatures()), ('regression', Ridge())])

In [13]:
print('Параметры лучшей модели:\n', select_model.best_params_)
r2_best_model = r2_score(y_test, select_model.predict(X_test))
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

Параметры лучшей модели:
 {'imputer': MinMaxScaler(), 'polynomial__degree': 2, 'regression': Ridge(), 'regression__alpha': 1.0}


In [14]:
results_regression

,model,task,R2
0,LR,task2,0.703496
1,Ridge,task2,0.698535
2,Lasso,task2,0.654746
3,Ridge_GridSearchCV,task3,0.712425
4,RidgeCV,task3,0.703428
5,Lasso_GridSearchCV,task3,0.712350
6,LassoCV,task3,0.703495
7,Ridge_StandardScaler,task4,0.703263
8,Ridge_MinMaxScaler,task4,0.701020
9,Lasso_StandardScaler,task4,0.627705


http://archive.ics.uci.edu/ml/datasets/Adult

In [15]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [16]:
### Ваш код ###
data.columns = data.columns.str.replace(' ', '')
data = data.replace(r'\s+','',regex=True)
data['class'].unique()
data.loc[(data['class'] == '<=50K'), 'target'] = 1
data['target'] = data['target'].fillna(0)
data['target'].unique()
X = data.drop(['target', 'class'],axis=1)
y = data['target']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [17]:
y.value_counts()

target
1.0    37155
0.0    11687
Name: count, dtype: int64

In [18]:
### Ваш код ###
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

Dummy = DummyClassifier(strategy="most_frequent")
Dummy.fit(X, y)

f1_most_frequent = f1_score(y, Dummy.predict(X))
acc_most_frequent = accuracy_score(y, Dummy.predict(X))
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

f1_most_frequent, acc_most_frequent

(0.8640999104619929, 0.7607182343065395)

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [19]:
data.info()
#пропусков не обнаружено

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             48842 non-null  int64  
 1   workclass       48842 non-null  object 
 2   fnlwgt          48842 non-null  int64  
 3   education       48842 non-null  object 
 4   education-num   48842 non-null  int64  
 5   marital-status  48842 non-null  object 
 6   occupation      48842 non-null  object 
 7   relationship    48842 non-null  object 
 8   race            48842 non-null  object 
 9   sex             48842 non-null  object 
 10  capital-gain    48842 non-null  int64  
 11  capital-loss    48842 non-null  int64  
 12  hours-per-week  48842 non-null  int64  
 13  native-country  48842 non-null  object 
 14  class           48842 non-null  object 
 15  target          48842 non-null  float64
dtypes: float64(1), int64(6), object(9)
memory usage: 6.0+ MB


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [20]:
### Ваш код ###
data_category = X.select_dtypes('object').columns
data_num = X.select_dtypes('number').columns
X[data_num]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
48837,39,215419,13,0,0,36
48838,64,321403,9,0,0,40
48839,38,374983,13,0,0,50
48840,44,83891,13,5455,0,40


In [21]:
X[data_category]

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
48837,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
48838,?,HS-grad,Widowed,?,Other-relative,Black,Male,United-States
48839,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
48840,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [22]:
X[data_category].describe(include='object')

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
count,48842,48842,48842,48842,48842,48842,48842,48842
unique,9,16,7,15,6,5,2,42
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,33906,15784,22379,6172,19716,41762,32650,43832


In [23]:
for col in data_category:
    one_hot = pd.get_dummies(X[col], prefix=col, drop_first=True, dtype=int)
    X = pd.concat((X.drop(col, axis=1), one_hot), axis=1)

In [24]:
X

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,215419,13,0,0,36,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48838,64,321403,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48839,38,374983,13,0,0,50,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48840,44,83891,13,5455,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [25]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
X[data_num] = min_max.fit_transform(X[data_num])
X = pd.concat((X.drop(data_num, axis=1), X[data_num]), axis=1)

In [26]:
X

,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,...,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959
1,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0.452055,0.048052,0.800000,0.000000,0.0,0.122449
2,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.287671,0.137581,0.533333,0.000000,0.0,0.397959
3,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0.493151,0.150486,0.400000,0.000000,0.0,0.397959
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0.150685,0.220635,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.301370,0.137428,0.800000,0.000000,0.0,0.357143
48838,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0.643836,0.209130,0.533333,0.000000,0.0,0.397959
48839,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.287671,0.245379,0.800000,0.000000,0.0,0.500000
48840,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.369863,0.048444,0.800000,0.054551,0.0,0.397959


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101, test_size=0.2)

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [29]:
X_test

,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,...,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
30658,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.342466,0.045160,0.533333,0.0,0.0,0.397959
25493,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.205479,0.055236,0.533333,0.0,0.0,0.397959
11136,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0.397260,0.143306,0.533333,0.0,0.0,0.500000
3814,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.109589,0.133741,0.800000,0.0,0.0,0.397959
743,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.123288,0.068034,0.666667,0.0,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0.397260,0.061120,0.866667,0.0,0.0,0.397959
3070,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.136986,0.195769,0.533333,0.0,0.0,0.397959
37753,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.013699,0.113193,0.533333,0.0,0.0,0.193878
22489,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0.273973,0.129586,0.533333,0.0,0.0,0.448980


In [30]:
#LogisticRegression
Log_Reg = LogisticRegression(max_iter=1000)
Log_Reg.fit(X_train, y_train)
Log_Reg_pred = Log_Reg.predict(X_test)
Log_Reg_acc =cross_val_score(Log_Reg, X_test, y_test, scoring='accuracy')
Log_Reg_f1 = cross_val_score(Log_Reg, X_test, y_test, scoring='f1')
Log_Reg_acc.mean(), Log_Reg_f1.mean()

(0.8403113389840369, 0.8979410995780605)

In [31]:
# KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train, y_train)
KNN_pred = KNN.predict(X_test.values)
KNN_acc = cross_val_score(KNN, X_test.values, y_test.values, scoring='accuracy')
KNN_f1 = cross_val_score(KNN, X_test.values, y_test.values, scoring='f1')
KNN_acc.mean(), KNN_f1.mean()

C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


(0.7633341561495555, 0.8318459993600277)

In [32]:
#LinearSVC
L_SVC = LinearSVC()
L_SVC.fit(X_train, y_train)
L_SVC_pred = L_SVC.predict(X_test)
L_SVC_acc =cross_val_score(L_SVC, X_test, y_test, scoring='accuracy')
L_SVC_f1 = cross_val_score(L_SVC, X_test, y_test, scoring='f1')
L_SVC_acc.mean(), L_SVC_f1.mean()

C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of 

(0.8486035970171077, 0.903216614437398)

In [33]:
### Ваш код ###
f1_LR = Log_Reg_f1.mean()
acc_LR = Log_Reg_acc.mean()
f1_KNN = KNN_f1.mean()
acc_KNN = KNN_acc.mean()
f1_SVM = L_SVC_f1.mean()
acc_SVM = L_SVC_acc.mean()
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

In [34]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.897941,0.840311
2,KNeighborsClassifier,task13,0.831846,0.763334
3,LinearSVC,task13,0.903217,0.848604


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('imputer', SimpleImputer(missing_values='?', 
                                                                                      strategy="most_frequent")), 
                                                            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first',
                                                                                      sparse_output=False))]), data_category),
                                          ('num', Pipeline([('scaler', MinMaxScaler())]), data_num)])

In [36]:
X_SI = data.drop(['target', 'class'],axis=1)
X_SI

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [37]:
X_SI = pd.DataFrame(preprocessor_imputer.fit_transform(X_SI))
X_SI

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.048052,0.800000,0.000000,0.0,0.122449
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.137581,0.533333,0.000000,0.0,0.397959
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.150486,0.400000,0.000000,0.0,0.397959
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.220635,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.137428,0.800000,0.000000,0.0,0.357143
48838,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.643836,0.209130,0.533333,0.000000,0.0,0.397959
48839,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.245379,0.800000,0.000000,0.0,0.500000
48840,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.369863,0.048444,0.800000,0.054551,0.0,0.397959


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X_SI, y, random_state=101, test_size=0.2)

In [39]:
#LogisticRegression
Log_Reg_SI = LogisticRegression(max_iter=1000)
Log_Reg_SI.fit(X_train, y_train)
Log_Reg_SI_pred = Log_Reg_SI.predict(X_test)
Log_Reg_SI_acc =cross_val_score(Log_Reg_SI, X_test, y_test, scoring='accuracy')
Log_Reg_SI_f1 = cross_val_score(Log_Reg_SI, X_test, y_test, scoring='f1')
Log_Reg_SI_acc.mean(), Log_Reg_SI_f1.mean()

(0.8401068403280574, 0.8978895057721354)

In [40]:
# KNeighborsClassifier
KNN_SI = KNeighborsClassifier(n_neighbors=2)
KNN_SI.fit(X_train, y_train)
KNN_SI_pred = KNN_SI.predict(X_test.values)
KNN_SI_acc =cross_val_score(KNN_SI, X_test.values, y_test.values, scoring='accuracy')
KNN_SI_f1 = cross_val_score(KNN_SI, X_test.values, y_test.values, scoring='f1')
KNN_SI_acc.mean(), KNN_SI_f1.mean()

(0.7650747531158267, 0.8332798894634577)

In [41]:
#LinearSVC
L_SVC_SI = LinearSVC()
L_SVC_SI.fit(X_train, y_train)
L_SVC_SI_pred = L_SVC_SI.predict(X_test)
L_SVC_SI_acc =cross_val_score(L_SVC_SI, X_test, y_test, scoring='accuracy')
L_SVC_SI_f1 = cross_val_score(L_SVC_SI, X_test, y_test, scoring='f1')
L_SVC_SI_acc.mean(), L_SVC_SI_f1.mean()

C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of 

(0.846761065174906, 0.9022212613402314)

In [42]:
f1_LR = Log_Reg_SI_f1.mean()
acc_LR = Log_Reg_SI_acc.mean()
f1_KNN = KNN_SI_f1.mean()
acc_KNN = KNN_SI_acc.mean()
f1_SVM = L_SVC_SI_f1.mean()
acc_SVM = L_SVC_SI_acc.mean()
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [43]:
data_del = data.copy()
data_del

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,1.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K,1.0
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K,1.0
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K,1.0
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K,1.0


In [44]:
data_del = data_del.replace({'?': np.nan}).dropna()
data_del

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,1.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48836,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K,1.0
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K,1.0
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K,1.0
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K,1.0


In [45]:
X_del = data_del.drop(['target', 'class'],axis=1)
y_del = data_del['target']
data_category = X_del.select_dtypes('object').columns
data_num = X_del.select_dtypes('number').columns
X_del.reset_index(drop=True, inplace=True)
y_del.reset_index(drop=True, inplace=True)

In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler

preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore', drop='first',
                                                                                      sparse_output=False))]), data_category),
                                          ('num', Pipeline([('scaler', MinMaxScaler())]), data_num)])

X_del = pd.DataFrame(preprocessor_imputer.fit_transform(X_del))
X_del

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.043350,0.800000,0.021740,0.0,0.397959
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.047274,0.800000,0.000000,0.0,0.122449
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.136877,0.533333,0.000000,0.0,0.397959
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.149792,0.400000,0.000000,0.0,0.397959
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.219998,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.219178,0.156895,0.800000,0.000000,0.0,0.397959
45218,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.136723,0.800000,0.000000,0.0,0.357143
45219,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.244762,0.800000,0.000000,0.0,0.500000
45220,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.369863,0.047666,0.800000,0.054551,0.0,0.397959


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X_del, y_del, random_state=101, test_size=0.2)

In [48]:
#LogisticRegression
Log_Reg_del = LogisticRegression(max_iter=1000)
Log_Reg_del.fit(X_train, y_train)
Log_Reg_del_pred = Log_Reg_del.predict(X_test)
Log_Reg_del_acc =cross_val_score(Log_Reg_del, X_test, y_test, scoring='accuracy')
Log_Reg_del_f1 = cross_val_score(Log_Reg_del, X_test, y_test, scoring='f1')
Log_Reg_del_acc.mean(), Log_Reg_del_f1.mean()

(0.8399115533443892, 0.8965323356789353)

In [49]:
# KNeighborsClassifier
KNN_del = KNeighborsClassifier(n_neighbors=2)
KNN_del.fit(X_train, y_train)
KNN_del_pred = KNN_del.predict(X_test.values)
KNN_del_acc =cross_val_score(KNN_del, X_test.values, y_test.values, scoring='accuracy')
KNN_del_f1 = cross_val_score(KNN_del, X_test.values, y_test.values, scoring='f1')
KNN_del_acc.mean(), KNN_del_f1.mean()

(0.7611940298507462, 0.8278834101639809)

In [50]:
#LinearSVC
L_SVC_del = LinearSVC()
L_SVC_del.fit(X_train, y_train)
L_SVC_del_pred = L_SVC_del.predict(X_test)
L_SVC_del_acc =cross_val_score(L_SVC_del, X_test, y_test, scoring='accuracy')
L_SVC_del_f1 = cross_val_score(L_SVC_del, X_test, y_test, scoring='f1')
L_SVC_del_acc.mean(), L_SVC_del_f1.mean()

C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ilyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of 

(0.8463239358761747, 0.9007597860982461)

In [51]:
f1_LR_del_missings = Log_Reg_del_f1.mean()
acc_LR_del_missings = Log_Reg_del_acc.mean()
f1_KNN_del_missings = KNN_del_f1.mean()
acc_KNN_del_missings = KNN_del_acc.mean()
f1_SVM_del_missings = L_SVC_del_f1.mean()
acc_SVM_del_missings = L_SVC_del_acc.mean()
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [52]:
data_SI = data.copy()
data_SI

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,1.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K,1.0
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K,1.0
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K,1.0
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K,1.0


In [53]:
X_SI = data_SI.drop(['target', 'class'],axis=1)
y_SI = data_SI['target']
data_category = X_SI.select_dtypes('object').columns
data_num = X_SI.select_dtypes('number').columns

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('imputer', SimpleImputer(missing_values='?', 
                                                                                      strategy="most_frequent")), 
                                                            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first',
                                                                                      sparse_output=False))]), data_category),
                                          ('num', Pipeline([('scaler', MinMaxScaler())]), data_num)])

In [55]:
X_SI = pd.DataFrame(preprocessor_imputer.fit_transform(X_SI))
X_SI

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.048052,0.800000,0.000000,0.0,0.122449
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.137581,0.533333,0.000000,0.0,0.397959
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.150486,0.400000,0.000000,0.0,0.397959
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.220635,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.137428,0.800000,0.000000,0.0,0.357143
48838,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.643836,0.209130,0.533333,0.000000,0.0,0.397959
48839,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.245379,0.800000,0.000000,0.0,0.500000
48840,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.369863,0.048444,0.800000,0.054551,0.0,0.397959


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X_SI, y, random_state=101, test_size=0.2)

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# RandomForestClassifier
RF = RandomForestClassifier(max_depth=2, random_state=0).fit(X_train, y_train)
RF_pred = RF.predict(X_test)
acc_RF = cross_val_score(RF, X_test, y_test, cv=3, scoring='accuracy')
f1_RF = cross_val_score(RF, X_test, y_test, cv=3, scoring='f1')
acc_RF.mean(), f1_RF.mean()

(0.7669160004898413, 0.8668509028804904)

In [58]:
# GradientBoostingClassifier
GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
GB_pred = GB.predict(X_test)
acc_GB = cross_val_score(GB, X_test, y_test, cv=3, scoring='accuracy')
f1_GB = cross_val_score(GB, X_test, y_test, cv=3, scoring='f1')
acc_GB.mean(), f1_GB.mean()

(0.8557682539490953, 0.9081883849546278)

In [59]:
f1_RF = f1_RF.mean()
acc_RF = acc_RF.mean()
f1_GB = f1_GB.mean()
acc_GB = acc_GB.mean()
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [60]:
from sklearn.exceptions import FitFailedWarning

import warnings

warnings.simplefilter("ignore", category=FitFailedWarning)

In [61]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer

X = X.replace('?', np.nan)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler()],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'preprocessor__cat__onehot__handle_unknown': ['ignore', 'error']
}

best_params = {}
f1_best = 0
acc_best = 0

for model_name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=2, 
        scoring={'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score)},
        refit='f1', 
        n_jobs=-1
    )
    
    grid_search.fit(X, y)
    
    best_params[model_name] = grid_search.best_params_
    f1_score_mean = grid_search.best_score_
    acc_score_mean = cross_val_score(grid_search.best_estimator_, X, y, cv=2, scoring='accuracy').mean()
    
    current_results = pd.DataFrame({
        'model': [model_name],
        'task': ['task17'],
        'f1': [f1_score_mean],
        'accuracy': [acc_score_mean]
    })
    
    if not current_results.empty:
        results_classification = pd.concat([results_classification, current_results], ignore_index=True)

print('Параметры лучшей модели:\n', best_params)
print(results_classification)

Параметры лучшей модели:
 {'RandomForestClassifier': {'preprocessor__cat__imputer__strategy': 'constant', 'preprocessor__cat__onehot__handle_unknown': 'error', 'preprocessor__num__imputer__strategy': 'most_frequent', 'preprocessor__num__scaler': MinMaxScaler()}, 'GradientBoostingClassifier': {'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__cat__onehot__handle_unknown': 'error', 'preprocessor__num__imputer__strategy': 'most_frequent', 'preprocessor__num__scaler': MinMaxScaler()}}
                                   model    task        f1  accuracy
0                    Most Frequent class  task10  0.864100  0.760718
1                     LogisticRegression  task13  0.897941  0.840311
2                   KNeighborsClassifier  task13  0.831846  0.763334
3                              LinearSVC  task13  0.903217  0.848604
4              LogisticRegression_impute  task14  0.897890  0.840107
5            KNeighborsClassifier_impute  task14  0.833280  0.765075
6        

In [62]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.897941,0.840311
2,KNeighborsClassifier,task13,0.831846,0.763334
3,LinearSVC,task13,0.903217,0.848604
4,LogisticRegression_impute,task14,0.897890,0.840107
5,KNeighborsClassifier_impute,task14,0.833280,0.765075
6,LinearSVC_impute,task14,0.902221,0.846761
7,LogisticRegression_delete_missings,task15,0.896532,0.839912
8,KNeighborsClassifier_delete_missings,task15,0.827883,0.761194
9,LinearSVC_delete_missings,task15,0.900760,0.846324
